In [1]:
%cd ../
%pwd

D:\Projects\IndiaTB\IndTB_ppa


'D:\\Projects\\IndiaTB\\IndTB_ppa'

In [14]:
import json
from ppa import Cascade
import numpy as np
import pandas as pd

In [13]:
locs = ['India', 'Delhi', 'Uttar Pradesh', 'Kerala']

In [24]:
mss = list()


for loc in locs:
    with open(f'out/cascade_2019/Cascade_{loc}.json', 'r') as f:
        cs = json.load(f)
    
    for i, c in enumerate(cs):
        cas = Cascade.parse_obj(c)
        cas.revive()
        
        dur = 1 / (cas.R_Succ_Tx + cas.R_Die_Tx + cas.R_LTFU_Tx)

        ms = {
            'Key': i,
            'Location': loc,
            'PPV_Pri': cas.PPV[2],
            'TxI_Pri': cas.P_TxI[2],
            'DurTx_Pri': dur[2],
            'TxTime_Pub': cas.PrevTx[0],
            'TxTime_Eng': cas.PrevTx[1],
            'TxTime_Pri': cas.PrevTx[2],
            'PriPub': cas.PrevTx[1:].sum() / cas.PrevTx[0]
        }
        
        mss.append(ms)
        
mss = pd.DataFrame(mss)

In [25]:
cas.P_TxI

array([0.98      , 0.95      , 0.65430401])

In [26]:
mss.to_csv('out/cascade_2019/ToMatchDS.csv')